In [ ]:
import sagemaker
import pandas as pd
import numpy as np
from ag_model import AutoGluonInferenceModel
from sagemaker.serializers import CSVSerializer
import os

# Deploy Endpoint

In [ ]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()

In [ ]:
model_dir = '.'
endpoint_name = sagemaker.utils.unique_name_from_base("sagemaker-autogluon-serving-trained-model")
endpoint_name

Upload the model we trained earlier

In [ ]:
model_data = sagemaker_session.upload_data(path=os.path.join(model_dir, 'model.tar.gz'), key_prefix=f'{endpoint_name}/models')
model_data

In [ ]:
model = AutoGluonInferenceModel(
    model_data=model_data, 
    role=role, 
    source_dir="scripts",
    entry_point="tabular_serve.py",
)

Deploy SageMaker inference endpoint (this might take a while)

In [ ]:
predictor = model.deploy(initial_instance_count=1, serializer = CSVSerializer(), instance_type="ml.m5.2xlarge")

# Using Endpoint for Inference

In [ ]:
!wget https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv -O data/test.csv

In [ ]:
df = pd.read_csv("data/test.csv")
df.head()

In [ ]:
data = df.drop(columns='class')[:100].values
preds = predictor.predict(data) # test inference with SDK

In [ ]:
preds[:10]

In [ ]:
p = pd.DataFrame({'preds': pd.DataFrame(preds)[0], 'actual': df['class'][:len(preds)]})
p.head()

In [ ]:
print(f'{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct')

# Cleanup

In [ ]:
# predictor.delete_model()
# predictor.delete_endpoint()